# API Data export for Griffin 
## 1. Import 

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import tweepy
#import dotenv
import os
import yaml
import datetime
from pprint import pprint
import time
import re

In [2]:
#Import Tweets
#all_user = pd.read_csv('users.csv', lineterminator='\n')
#all_tweets = pd.read_csv('tweets.csv', lineterminator='\n')
all_user= pd.read_pickle("./users.pkl")
all_tweets = pd.read_pickle("./tweets.pkl")


In [3]:
all_user.head()

,created_at,description,author_id,location,name,pinned_tweet_id,profile_image_url,protected,url,username,verified,followers_count,following_count,tweet_count,listed_count
0,2015-01-21 22:00:08+00:00,"Greta Fan, Umweltschützer, Tierschützer, Marxi...",2990631651,"Wien, Österreich",Freitagsdemonstrant,NaN,https://pbs.twimg.com/profile_images/144530169...,False,https://t.co/UXz3ohzZmK,Freitagsdemonst,False,54,63,1854,1
1,2021-08-09 19:59:46+00:00,"Wir sind die #LetzteGeneration, die den völlig...",1424822661474144256,Berlin,Letzte Generation,1.539137e+18,https://pbs.twimg.com/profile_images/150151470...,False,https://t.co/bwmWM5KdNp,AufstandLastGen,False,13556,234,2873,84
2,2021-04-02 04:39:19+00:00,"Klimaaktivist, weil wir die letzte Generation ...",1377842769117519873,"Wien, Österreich",KlimaApp - #letztegeneration,1.477782e+18,https://pbs.twimg.com/profile_images/153210162...,False,https://t.co/jirYKZD0oH,klimaapp,False,338,1819,2910,3
3,2009-05-06 10:08:02+00:00,"Bundestagsabgeordneter für den Wahlkreis 61, S...",38150247,"Berlin, Deutschland",Olaf Scholz,NaN,https://pbs.twimg.com/profile_images/153453840...,False,https://t.co/pDZ6VOLHnA,OlafScholz,True,555303,1440,6625,2648
4,2020-05-06 15:06:12+00:00,Greta's Rottweiler. Climate Deniers Beware. Tw...,1258050319356919810,"Vienna, Austria",Martha #StandWithUkraine 🇺🇦,NaN,https://pbs.twimg.com/profile_images/148976129...,False,,Martha_LastGen,False,713,240,3331,0


In [4]:
all_tweets.head()

,author_id,conversation_id,created_at,entities,geo,id,text,retweet_count,reply_count,like_count,quote_count,media_keys,username
0,2990631651,1539136601640603650,2022-06-21 08:51:55+00:00,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",{'place_id': '9f659d51e5c5deae'},1539169230825857025,@AufstandLastGen @klimaapp @OlafScholz Warum w...,0,0,1,0,7_1539167493276721153,Freitagsdemonst
1,1238554146749460483,1539168078306607108,2022-06-21 08:47:20+00:00,NaN,NaN,1539168078306607108,https:tcoUcjZjZdzAn\n\nFridaysForFuture Hambur...,0,0,0,0,NaN,CornelsSelke
2,1350905394395275270,1539163851840753664,2022-06-21 08:30:33+00:00,NaN,NaN,1539163851840753664,FollowTheMoney\nKlimahysterie \nKlimawandel \n...,0,0,0,0,NaN,kryptoschland
3,18720161,1538593189321486337,2022-06-21 08:28:09+00:00,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",NaN,1539163250729926658,"@HeinrichU @Scheife @florianklenk Oh, muss ich...",0,0,0,0,NaN,fhohenauer
4,321107517,1539162009790517249,2022-06-21 08:23:14+00:00,NaN,NaN,1539162009790517249,"Yesterday in Abadan Iran, the temperature rais...",0,0,0,0,3_1539161705196015616,mind2merkabah


## 2. User/ Nodes Mapping

In [12]:
nodes = all_user.drop(['created_at', 'description','location', 'pinned_tweet_id', 'profile_image_url', 'protected', 'url', 'verified' ], axis = 1)
nodes = nodes.rename(columns = {'following_count': 'friends_count', 'tweet_count' : 'statuses_count'}, errors="raise")
nodes

,author_id,name,username,followers_count,friends_count,statuses_count,listed_count
0,2990631651,Freitagsdemonstrant,Freitagsdemonst,54,63,1854,1
1,1424822661474144256,Letzte Generation,AufstandLastGen,13556,234,2873,84
2,1377842769117519873,KlimaApp - #letztegeneration,klimaapp,338,1819,2910,3
3,38150247,Olaf Scholz,OlafScholz,555303,1440,6625,2648
4,1258050319356919810,Martha #StandWithUkraine 🇺🇦,Martha_LastGen,713,240,3331,0
...,...,...,...,...,...,...,...
296,273769218,Robert Miller,winexus,65,131,1195,7
297,330976924,Kronen Zeitung,krone_at,83583,48,135737,557
298,1431661233233612806,Human Powered Messages for The Planet,MessagesToCOP28,39,15,55,2
299,746544798228676608,Kirsten Schölzel,ElseStratten,7,23,6591,0


## 3. Tweet/ Edges Mapping

In [5]:
# exclude all Tweets whitout mentions
all_tweets_entities_null = all_tweets 
all_tweets_entities_null=all_tweets_entities_null[all_tweets_entities_null['entities'].isna()]
all_tweets_entities_null

,author_id,conversation_id,created_at,entities,geo,id,text,retweet_count,reply_count,like_count,quote_count,media_keys,username
1,1238554146749460483,1539168078306607108,2022-06-21 08:47:20+00:00,NaN,NaN,1539168078306607108,https:tcoUcjZjZdzAn\n\nFridaysForFuture Hambur...,0,0,0,0,NaN,CornelsSelke
2,1350905394395275270,1539163851840753664,2022-06-21 08:30:33+00:00,NaN,NaN,1539163851840753664,FollowTheMoney\nKlimahysterie \nKlimawandel \n...,0,0,0,0,NaN,kryptoschland
4,321107517,1539162009790517249,2022-06-21 08:23:14+00:00,NaN,NaN,1539162009790517249,"Yesterday in Abadan Iran, the temperature rais...",0,0,0,0,3_1539161705196015616,mind2merkabah
8,1523320630090899457,1539152802957729793,2022-06-21 07:46:38+00:00,NaN,NaN,1539152802957729793,Temperatur am 20062022: 159 C\n\nVergleich: 19...,1,1,1,0,NaN,history_climate
9,1523320630090899457,1538778831716237312,2022-06-20 07:00:37+00:00,NaN,NaN,1538778831716237312,Temperatur am 19062022: 249 C\n\nVergleich: 19...,0,1,1,0,NaN,history_climate
...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,1516003236095963140,1538779227922784257,2022-06-20 07:02:11+00:00,NaN,NaN,1538779227922784257,"kohlekraftwerk wird aktiviert, da für haben di...",1,0,0,0,NaN,44_sevgili
179,273769218,1538729834955673600,2022-06-20 03:45:55+00:00,NaN,NaN,1538729834955673600,As we hiked into the clouds on the Davis Creek...,2,1,3,0,3_1538729804576329728,winexus
181,1431661233233612806,1538714781661528064,2022-06-20 02:46:06+00:00,NaN,NaN,1538714781661528064,"To change everything, we need EVERYONE Let us ...",0,0,0,0,3_1538713870847471617,MessagesToCOP28
182,746544798228676608,1538693858351005698,2022-06-20 01:22:58+00:00,NaN,NaN,1538693858351005698,In Grömitz wird die Seebrücke Tag und Nacht be...,0,0,0,0,NaN,ElseStratten


In [6]:
# Save all Tweet with mention
all_tweets_mention = all_tweets
all_tweets_mention.dropna(subset=['entities'], inplace = True)
all_tweets_mention

,author_id,conversation_id,created_at,entities,geo,id,text,retweet_count,reply_count,like_count,quote_count,media_keys,username
0,2990631651,1539136601640603650,2022-06-21 08:51:55+00:00,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",{'place_id': '9f659d51e5c5deae'},1539169230825857025,@AufstandLastGen @klimaapp @OlafScholz Warum w...,0,0,1,0,7_1539167493276721153,Freitagsdemonst
3,18720161,1538593189321486337,2022-06-21 08:28:09+00:00,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",NaN,1539163250729926658,"@HeinrichU @Scheife @florianklenk Oh, muss ich...",0,0,0,0,NaN,fhohenauer
5,1214456673605165056,1539161285996294144,2022-06-21 08:20:21+00:00,"{'mentions': [{'start': 13, 'end': 16, 'userna...",NaN,1539161285996294144,"Ahead of the @G7 summit, Rise up movement and ...",11,2,17,0,3_1539161277825679360,Riseupmovt
6,1433079473344221185,1539154365461762048,2022-06-21 07:52:51+00:00,"{'mentions': [{'start': 28, 'end': 43, 'userna...",NaN,1539154365461762048,Vielleicht möchte Neubauer @Luisamneubauer jet...,0,0,0,0,NaN,KaleuDer
7,1433079473344221185,1538871316446687237,2022-06-21 07:52:41+00:00,"{'mentions': [{'start': 0, 'end': 9, 'username...",NaN,1539154324588306432,@ZDFheute Vielleicht möchte Neubauer @Luisamne...,0,0,0,0,NaN,KaleuDer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,1375903974356889600,1538793668747960320,2022-06-20 07:59:34+00:00,"{'mentions': [{'start': 77, 'end': 87, 'userna...",NaN,1538793668747960320,Ich werde meinen Tankrabatt komplett an Friday...,0,1,4,0,NaN,_RedKiteNRW
175,1525742812628766721,1538780981963984896,2022-06-20 07:09:09+00:00,"{'mentions': [{'start': 61, 'end': 72, 'userna...",NaN,1538780981963984896,Climate justice Racial justice \nFFFmapa Frid...,42,1,110,0,3_1538780979044704257,gracy_ssanyu
177,1485857166405820421,1536703873796259842,2022-06-20 05:35:58+00:00,"{'mentions': [{'start': 0, 'end': 14, 'usernam...",NaN,1538757528917618688,@olafinderbeek Das nehme ich Ihnen absolut nic...,16,3,105,0,NaN,klaus_wiens
178,22814467,1538582419409969155,2022-06-20 04:15:48+00:00,"{'mentions': [{'start': 0, 'end': 12, 'usernam...",NaN,1538737357230424065,@fred_Noethe @midnite8lue @notruf_NRW Ob Frida...,0,0,5,0,NaN,AndrBangemann


In [8]:
# Unpack Mentions in new Column 
names = []

for item in all_tweets_mention['entities']:
    names.append(item["mentions"])
    
all_tweets_mention['mentions'] = names
all_tweets_mention.head()

,author_id,conversation_id,created_at,entities,geo,id,text,retweet_count,reply_count,like_count,quote_count,media_keys,username,mentions
0,2990631651,1539136601640603650,2022-06-21 08:51:55+00:00,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",{'place_id': '9f659d51e5c5deae'},1539169230825857025,@AufstandLastGen @klimaapp @OlafScholz Warum w...,0,0,1,0,7_1539167493276721153,Freitagsdemonst,"[{'start': 0, 'end': 16, 'username': 'Aufstand..."
3,18720161,1538593189321486337,2022-06-21 08:28:09+00:00,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",NaN,1539163250729926658,"@HeinrichU @Scheife @florianklenk Oh, muss ich...",0,0,0,0,NaN,fhohenauer,"[{'start': 0, 'end': 10, 'username': 'Heinrich..."
5,1214456673605165056,1539161285996294144,2022-06-21 08:20:21+00:00,"{'mentions': [{'start': 13, 'end': 16, 'userna...",NaN,1539161285996294144,"Ahead of the @G7 summit, Rise up movement and ...",11,2,17,0,3_1539161277825679360,Riseupmovt,"[{'start': 13, 'end': 16, 'username': 'G7', 'i..."
6,1433079473344221185,1539154365461762048,2022-06-21 07:52:51+00:00,"{'mentions': [{'start': 28, 'end': 43, 'userna...",NaN,1539154365461762048,Vielleicht möchte Neubauer @Luisamneubauer jet...,0,0,0,0,NaN,KaleuDer,"[{'start': 28, 'end': 43, 'username': 'Luisamn..."
7,1433079473344221185,1538871316446687237,2022-06-21 07:52:41+00:00,"{'mentions': [{'start': 0, 'end': 9, 'username...",NaN,1539154324588306432,@ZDFheute Vielleicht möchte Neubauer @Luisamne...,0,0,0,0,NaN,KaleuDer,"[{'start': 0, 'end': 9, 'username': 'ZDFheute'..."


In [9]:
# Create single row for each mention
all_tweets_mentions_ids = all_tweets_mention
all_tweets_mentions_ids = all_tweets_mentions_ids.explode('mentions')

all_tweets_mentions_ids.head()

,author_id,conversation_id,created_at,entities,geo,id,text,retweet_count,reply_count,like_count,quote_count,media_keys,username,mentions
0,2990631651,1539136601640603650,2022-06-21 08:51:55+00:00,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",{'place_id': '9f659d51e5c5deae'},1539169230825857025,@AufstandLastGen @klimaapp @OlafScholz Warum w...,0,0,1,0,7_1539167493276721153,Freitagsdemonst,"{'start': 0, 'end': 16, 'username': 'AufstandL..."
0,2990631651,1539136601640603650,2022-06-21 08:51:55+00:00,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",{'place_id': '9f659d51e5c5deae'},1539169230825857025,@AufstandLastGen @klimaapp @OlafScholz Warum w...,0,0,1,0,7_1539167493276721153,Freitagsdemonst,"{'start': 17, 'end': 26, 'username': 'klimaapp..."
0,2990631651,1539136601640603650,2022-06-21 08:51:55+00:00,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",{'place_id': '9f659d51e5c5deae'},1539169230825857025,@AufstandLastGen @klimaapp @OlafScholz Warum w...,0,0,1,0,7_1539167493276721153,Freitagsdemonst,"{'start': 27, 'end': 38, 'username': 'OlafScho..."
0,2990631651,1539136601640603650,2022-06-21 08:51:55+00:00,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",{'place_id': '9f659d51e5c5deae'},1539169230825857025,@AufstandLastGen @klimaapp @OlafScholz Warum w...,0,0,1,0,7_1539167493276721153,Freitagsdemonst,"{'start': 260, 'end': 275, 'username': 'Martha..."
0,2990631651,1539136601640603650,2022-06-21 08:51:55+00:00,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",{'place_id': '9f659d51e5c5deae'},1539169230825857025,@AufstandLastGen @klimaapp @OlafScholz Warum w...,0,0,1,0,7_1539167493276721153,Freitagsdemonst,"{'start': 276, 'end': 290, 'username': 'origin..."


In [10]:
# Save mentions username as dst_name 
usernames = []

for item in all_tweets_mentions_ids['mentions']:
    usernames.append(item['username'])
    
all_tweets_mentions_ids['dst_name'] = usernames
all_tweets_mentions_ids.head()

,author_id,conversation_id,created_at,entities,geo,id,text,retweet_count,reply_count,like_count,quote_count,media_keys,username,mentions,dst_name
0,2990631651,1539136601640603650,2022-06-21 08:51:55+00:00,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",{'place_id': '9f659d51e5c5deae'},1539169230825857025,@AufstandLastGen @klimaapp @OlafScholz Warum w...,0,0,1,0,7_1539167493276721153,Freitagsdemonst,"{'start': 0, 'end': 16, 'username': 'AufstandL...",AufstandLastGen
0,2990631651,1539136601640603650,2022-06-21 08:51:55+00:00,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",{'place_id': '9f659d51e5c5deae'},1539169230825857025,@AufstandLastGen @klimaapp @OlafScholz Warum w...,0,0,1,0,7_1539167493276721153,Freitagsdemonst,"{'start': 17, 'end': 26, 'username': 'klimaapp...",klimaapp
0,2990631651,1539136601640603650,2022-06-21 08:51:55+00:00,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",{'place_id': '9f659d51e5c5deae'},1539169230825857025,@AufstandLastGen @klimaapp @OlafScholz Warum w...,0,0,1,0,7_1539167493276721153,Freitagsdemonst,"{'start': 27, 'end': 38, 'username': 'OlafScho...",OlafScholz
0,2990631651,1539136601640603650,2022-06-21 08:51:55+00:00,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",{'place_id': '9f659d51e5c5deae'},1539169230825857025,@AufstandLastGen @klimaapp @OlafScholz Warum w...,0,0,1,0,7_1539167493276721153,Freitagsdemonst,"{'start': 260, 'end': 275, 'username': 'Martha...",Martha_LastGen
0,2990631651,1539136601640603650,2022-06-21 08:51:55+00:00,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",{'place_id': '9f659d51e5c5deae'},1539169230825857025,@AufstandLastGen @klimaapp @OlafScholz Warum w...,0,0,1,0,7_1539167493276721153,Freitagsdemonst,"{'start': 276, 'end': 290, 'username': 'origin...",originalcmvie


In [11]:
edges_mentions = all_tweets_mentions_ids 
edges_mentions = all_tweets_mentions_ids.drop(['conversation_id', 'entities','geo', 'id', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'media_keys','mentions'], axis = 1)
edges_mentions = edges_mentions.rename(columns = {'author_id': 'src_id_str', 'username' : 'src_name'})
edges_mentions

,src_id_str,created_at,text,src_name,dst_name
0,2990631651,2022-06-21 08:51:55+00:00,@AufstandLastGen @klimaapp @OlafScholz Warum w...,Freitagsdemonst,AufstandLastGen
0,2990631651,2022-06-21 08:51:55+00:00,@AufstandLastGen @klimaapp @OlafScholz Warum w...,Freitagsdemonst,klimaapp
0,2990631651,2022-06-21 08:51:55+00:00,@AufstandLastGen @klimaapp @OlafScholz Warum w...,Freitagsdemonst,OlafScholz
0,2990631651,2022-06-21 08:51:55+00:00,@AufstandLastGen @klimaapp @OlafScholz Warum w...,Freitagsdemonst,Martha_LastGen
0,2990631651,2022-06-21 08:51:55+00:00,@AufstandLastGen @klimaapp @OlafScholz Warum w...,Freitagsdemonst,originalcmvie
...,...,...,...,...,...
178,22814467,2022-06-20 04:15:48+00:00,@fred_Noethe @midnite8lue @notruf_NRW Ob Frida...,AndrBangemann,fred_Noethe
178,22814467,2022-06-20 04:15:48+00:00,@fred_Noethe @midnite8lue @notruf_NRW Ob Frida...,AndrBangemann,midnite8lue
178,22814467,2022-06-20 04:15:48+00:00,@fred_Noethe @midnite8lue @notruf_NRW Ob Frida...,AndrBangemann,notruf_NRW
180,1175693805544255488,2022-06-20 02:47:33+00:00,@krone_at What would @GretaThunberg say Friday...,Roberto22117551,krone_at


In [14]:
edges_mentions_src = edges_mentions 
edges_mentions_src = edges_mentions_src.merge(nodes, how = 'inner', left_on='src_name', right_on='username')
edges_mentions_dst = edges_mentions_src.merge(nodes, how = 'inner', left_on='dst_name', right_on='username')
edges_mentions_dst = edges_mentions_dst.drop(['listed_count_y', 'statuses_count_y', 'friends_count_y', 'followers_count_y', 'author_id_x', 'username_x', 'username_y' ], axis = 1)
edges_mentions_dst = edges_mentions_dst.rename(columns = {'name_x': 'src_screen_name', 'followers_count_x' : 'src_followers_count', 'friends_count_x': 'src_friends_count', 'statuses_count_x': 'src_statuses_count', 'listed_count_x': 'src_listed_count', 'author_id_y':'dst_id_str', 'name_y': 'dst_screen_name',  })
edges_mentions_dst

,src_id_str,created_at,text,src_name,dst_name,src_screen_name,src_followers_count,src_friends_count,src_statuses_count,src_listed_count,dst_id_str,dst_screen_name
0,2990631651,2022-06-21 08:51:55+00:00,@AufstandLastGen @klimaapp @OlafScholz Warum w...,Freitagsdemonst,AufstandLastGen,Freitagsdemonstrant,54,63,1854,1,1424822661474144256,Letzte Generation
1,2990631651,2022-06-21 08:51:55+00:00,@AufstandLastGen @klimaapp @OlafScholz Warum w...,Freitagsdemonst,klimaapp,Freitagsdemonstrant,54,63,1854,1,1377842769117519873,KlimaApp - #letztegeneration
2,2990631651,2022-06-21 08:51:55+00:00,@AufstandLastGen @klimaapp @OlafScholz Warum w...,Freitagsdemonst,OlafScholz,Freitagsdemonstrant,54,63,1854,1,38150247,Olaf Scholz
3,2990631651,2022-06-21 08:51:55+00:00,@AufstandLastGen @klimaapp @OlafScholz Warum w...,Freitagsdemonst,Martha_LastGen,Freitagsdemonstrant,54,63,1854,1,1258050319356919810,Martha #StandWithUkraine 🇺🇦
4,2990631651,2022-06-21 08:51:55+00:00,@AufstandLastGen @klimaapp @OlafScholz Warum w...,Freitagsdemonst,originalcmvie,Freitagsdemonstrant,54,63,1854,1,838875643601379332,Critical Mass Wien
...,...,...,...,...,...,...,...,...,...,...,...,...
306,1485857166405820421,2022-06-20 05:35:58+00:00,@olafinderbeek Das nehme ich Ihnen absolut nic...,klaus_wiens,olafinderbeek,Klaus Wiens 🇺🇦,17,62,609,0,913327957602693121,Olaf in der Beek MdB
307,22814467,2022-06-20 04:15:48+00:00,@fred_Noethe @midnite8lue @notruf_NRW Ob Frida...,AndrBangemann,fred_Noethe,Andreas Bangemann,1251,712,6461,103,4011592815,Fred Nöthe
308,22814467,2022-06-20 04:15:48+00:00,@fred_Noethe @midnite8lue @notruf_NRW Ob Frida...,AndrBangemann,midnite8lue,Andreas Bangemann,1251,712,6461,103,283474794,"Midnite Blue, Bürgerin der Erde 🌍 😷💉💉💉"
309,22814467,2022-06-20 04:15:48+00:00,@fred_Noethe @midnite8lue @notruf_NRW Ob Frida...,AndrBangemann,notruf_NRW,Andreas Bangemann,1251,712,6461,103,1486317616939548680,Notruf NRW


## 3. Export Nodes and Edges 

In [ ]:
edges_mentions_dst.to_csv("edges.csv", index=False)
nodes.to_csv("nodes.csv", index=False)